# Lab 05 - Data Modeling and Visualisation

We have spent a bit of time discussion data handling, manipulation, and collection. We are going to get a little more into analyses of data now.

A *model* is a simplified representation of reality. A *mathematical model* is a presentation of reality using numbers.

We use models for three purposes: 1) we want to summarise data, 2) we want to replicate the working of the world with existing data, and 3) we want to forecast the future from past data.

Modeling, in a sense, is a data reduction process. We are taking lots of existing data and reducing it to a few key metrics. Models are difficult to create because the real world is messy. To quote statistician George Box: "All models are wrong but some are useful."

In this lab, we'll work with the [More Tweets, More Votes](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0079449) (MTMV) data (the original paper is [here](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0079449), if you're curious), and the Iris data we have been using in lab. The MTMV paper suggests that if a politician gets more tweets then they are more likely to get a higher vote share.

There are many different variables in the MTMV model. The most important variables are the *dependent variable* (that is the variable which we are trying to explain). In this case, this is vote share (<code>vote_share</code>). The paper uses the percentage of Republican vote share. The variables which may explain variation in the dependent variable are called the *independent variables*. In the MTMV model there are many different variables which may explain vote share, including:

- Incumbency
- Ideological leaning
- Age
- Education level
- Gender
- Race
- Media markets

Most critically, however, we want to measure whether Twitter mention share (<code>mshare</code>) has an effect on vote share. Again, the variable will use Republicans as a reference group. We will also pay special attention to incumbency (<code>rep_inc</code>) because this is often the best predictor of who wins an election.

First we'll load the the MTMV data. The argument <code>index_col</code> defines one of the existing columns in the file as an index column.

In [ ]:
import pandas as pd
import numpy as np
df_mtmv = pd.read_csv("data/mtmv_data_10_12.csv", index_col = 0)

## drop those columns which do not have vote_shape, mshare, or rep_inc
df_mtmv = df_mtmv.dropna(subset = ['vote_share', 'mshare', 'rep_inc'])

In the past lab we described the data frame by understanding the structure of the data. We took simple measures of central tendency and dispersion. Now we can divide those up by a third variable.

To do this, we use a <code>groupby</code> which allows us to group by a variable or set of variables and apply some operation across them.

In [ ]:
## vote share and mention share mean 
## by Republican incumbency
gr_mtmv = df_mtmv.groupby('rep_inc')
gr_mtmv[['vote_share', 'mshare']].mean()

In [ ]:
## vote share and mention share standard deviation 
## by Republican incumbency
gr_mtmv[['vote_share', 'mshare']].std()

Now we can use a metric like [Pearson correlation](https://en.wikipedia.org/wiki/Pearson_product-moment_correlation_coefficient) to summarise the associated between two variables. Pearson correlation is measure of the linear dependence between two variables X and Y. It ranges from [-1, 1].

In [ ]:
from scipy.stats.stats import pearsonr

pearsonr(df_mtmv['mshare'], df_mtmv['vote_share'])

This returns two values in a tuple. The first is the Pearson correlation itself. The second is what's called a [p-value](https://en.wikipedia.org/wiki/P-value). The correlation is said to be *statistically significant* if its p-value is below a certain threshold, usually 0.05.

Lastly, we can build a statistical model which attempts to summarise the data into a number of different metrics. This isn't a statistics course, so we won't get too much into statistics and the interpretation of statistical models. 

For this lab, we will be using *linear regression* to build a model for these data. Roughly, linear regression attempts to draw a trend line across multiple variables to fit the data. The regression procedure will create *coefficients* which measure the effect of any particular variable. Like with correlation, the coefficient is said to be *statistically significant* if its p-value is below a certain threshold, usually 0.05.

In [ ]:
from statsmodels.formula.api import ols

model = ols("vote_share ~ rep_inc + mshare + pct_white + \
            pct_college + med_hhinc + pct_female", df_mtmv).fit()
model.summary()

### Exercise 1

1. Load the iris dataset (it will be in the same place as the previous lab, 'data/iris.csv'). 
2. Take the mean and standard deviation of the sepal length and sepal width, grouped by species.
3. Calculate the correlation between sepal length and petal length.

## Visualisation

Visualisation is a body of methods to present data visually. There are three main reasons to visualise: 1) explore data, 2) confirm a model, and 3) present results. Visualisation can highlight some important variations in a dataset which may be obscured by only focusing on summary statistics or by using statistical models.

For instance, look below at tthe example of Anscombe quartet. The data are dispersed in various configurations. But the mean and linear trend line are the same in all.

![](img/anscombe.png)

To begin, we'll import all the necessary modules for visualisation. We use the <code>matplotlib</code> module to visualise. We also will use <code>%matplotlib inline</code> to indicate that we want graphs to appear in our notebook. Lastly we'll use the ggplot style to make our graphs appear a little more visually appealing. 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')

### Univariate visualisations

Univariate visualisations allow us to understand single variables. They let us get beyond looking only at the mean, median, and standard deviations of these variables. It should be one of the first parts of exploring a dataset.

The first method we can look at is the histogram. This plots how many values of a variable appear in separate equal-sized bins. So for instance, in a variable that represent percentage like <code>mshare</code>, values vary from 0 to 100. So we'll typically see bins of size 10.

In [ ]:
df_mtmv['mshare'].hist() 

We can compare different variables in a histogram side-by-side by subsetting the data frame.

In [ ]:
df_mtmv[['mshare', 'vote_share']].hist()

Another way of doing this is to overlay the different variables on top of each other in the histogram. We pass the argument <code>alpha</code> and set it to a 0.5 (0 is completely transparent, 1 is completely opaque) so we can see the overlap.

In [ ]:
df_mtmv[['mshare', 'vote_share', 'pct_white']].plot.hist(alpha = 0.5)

### Bivariate and multivariable visualisations

Bivariable and multivariable visualisations allow us to see the relationship between two or more variables. It is one of the first parts of model exploration. A basic *scatter plot* shows how one variable changes with the other. So for instance, we want to show how the dependent variable changes with the main independent variable <code>mshare</code>. We would put <code>mshare</code> on the x-axis and <code>vote_share</code> on the y-axis.

In [ ]:
df_mtmv.plot.scatter(x = 'mshare', y = 'vote_share')

To add a third variable, we can add colour which indicates which observations take on a particular value. So say we want to also see which observations had Republican incumbents. We do this in two lines. We first will plot the Republican incumbents. We do this by selecting only rows which have Republican incumbents and colouring them Red. We then select only the rows which have non-Republican incumbents and colour them Blue.

In [ ]:
ax = df_mtmv[df_mtmv['rep_inc'] == 1].plot.scatter(x = 'mshare', y = 'vote_share', color = 'Red', label = 'Rep')
df_mtmv[df_mtmv['rep_inc'] == 0].plot.scatter(x = 'mshare', y = 'vote_share', color = 'Blue', label = 'Dem', ax = ax)

### Exercise 2

1. Plot the histograms of both sepal length and sepal width.
2. Plot the histograms of sepal length and sepal width overlaid on each other.
3. Plot a scatter plot of sepal length and sepal width.
4. Plot a scatter plot of sepal length and sepal width, where virginica is Green, setosa is Red, and versicolor is Blue.